In [8]:
import os
import torch
import torch.nn as nn
import sys
import numpy as np
import torch.optim as optim

sys.path.append("../LMSCNet")
from LMSCNet.common.seed import seed_all
from LMSCNet.common.config import CFG
from LMSCNet.common.dataset import get_dataset
from LMSCNet.common.model import get_model
from LMSCNet.common.logger import get_logger
from LMSCNet.common.optimizer import build_optimizer, build_scheduler
from LMSCNet.common.io_tools import dict_to
from LMSCNet.common.metrics import Metrics
import LMSCNet.common.checkpoint as checkpoint

from Data.dataset import CarlaDataset, collate_fn_test
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
import torchvision.models as models
from torchmetrics import JaccardIndex

import open3d as o3d
import time
import numpy as np
import matplotlib.pyplot as plt
import os
import json
import pdb
from PIL import Image
import psutil

In [9]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
dtype = torch.float32  # Tensor type to be used

from LMSCNet.models.LMSCNet import LMSCNet

# Calculated acoording to train dataset
class_frequencies = np.array([
    0.00,
    6.29,
    1.86,
    0.03,
    0.16,
    0.67,
    0.78,
    50.25,
    17.21,
    2.63,
    5.89,
    4.46,
    0.03,
    0.00,
    1.14,
    0.17,
    0.00,
    3.07,
    0.01,
    0.71,
    0.27,
    0.01,
    4.35,
])

model = LMSCNet(class_num=23, input_dimensions=[128, 128, 8], class_frequencies=class_frequencies)

optimizer = optim.Adam(model.get_parameters(), lr=0.001, betas=(0.9, 0.999))

# Moving optimizer and model to used device
model = model.to(device=device)
for state in optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.to(device)

print(model)

LMSCNet(
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (Encoder_block1): Sequential(
    (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU()
  )
  (Encoder_block2): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): Conv2d(128, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): ReLU()
    (3): Conv2d(192, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
  )
  (Encoder_block3): Sequential(
    (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (1): Conv2d(192, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): ReLU()
    (3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
  )
  (Encoder_block4): Sequential(
    (0): MaxPool2d(kernel_size=2, stride

In [10]:
data_dir = "./Data/Scenes/Cartesian/Train"

coordinate_type = "cartesian"
cylindrical = coordinate_type=="cylindrical"
T = 1
B = 4 # Matching paper

carla_ds = CarlaDataset(directory=data_dir, device=device, num_frames=T, cylindrical=cylindrical)
dataloader = DataLoader(carla_ds, batch_size=B, shuffle=True, collate_fn=collate_fn_test)

In [11]:
LABEL_COLORS = np.array([
    (255, 255, 255), # None
    (70, 70, 70),    # Building
    (100, 40, 40),   # Fences
    (55, 90, 80),    # Other
    (255, 255, 0),   # Pedestrian
    (153, 153, 153), # Pole
    (157, 234, 50),  # RoadLines
    (0, 0, 255),  # Road
    (255, 255, 255),  # Sidewalk
    (0, 155, 0),  # Vegetation
    (255, 0, 0),     # Vehicle
    (102, 102, 156), # Wall
    (220, 220, 0),   # TrafficSign
    (70, 130, 180),  # Sky
    (255, 255, 255),     # Ground
    (150, 100, 100), # Bridge
    (230, 150, 140), # RailTrack
    (180, 165, 180), # GuardRail
    (250, 170, 30),  # TrafficLight
    (110, 190, 160), # Static
    (170, 120, 50),  # Dynamic
    (45, 60, 150),   # Water
    (145, 170, 100), # Terrain
]) / 255.0 # normalize each channel [0-1] since is what Open3D uses

def visualize_preds(probs, min_dim, max_dim, num_samples, cylindrical=True, min_thresh=0.75):
    preds = np.argmax(probs, axis=3)
    max_probs = np.amax(probs, axis=3)
    intervals = (max_dim - min_dim) / preds.shape
    
    
    x = np.linspace(min_dim[0], max_dim[0], num=preds.shape[0]) + intervals[0] / 2
    y = np.linspace(min_dim[1], max_dim[1], num=preds.shape[1]) + intervals[1] / 2
    z = np.linspace(min_dim[2], max_dim[2], num=preds.shape[2]) + intervals[2] / 2
    xv, yv, zv = np.meshgrid(x, y, z, indexing="ij")

    valid_cells = max_probs > min_thresh 
    valid_x = xv[valid_cells]
    valid_y = yv[valid_cells]
    valid_z = zv[valid_cells]
    labels = preds[valid_cells]

    valid_points = np.stack((valid_x, valid_y, valid_z)).T
    non_free = labels != 0
    valid_points = valid_points[non_free, :]
    labels = labels[non_free]

    # Fill in voxels
    N, __ = valid_points.shape
    new_points = np.random.uniform((valid_points - intervals / 2).reshape(N, 3, 1),
                                   (valid_points + intervals / 2).reshape(N, 3, 1),
                                   (N, 3, num_samples))
    new_labels = np.zeros((N, num_samples), dtype=np.uint32)
    labels = (new_labels + labels.reshape(-1, 1)).reshape(-1)
    valid_points = np.transpose(new_points, (0, 2, 1)).reshape(-1, 3)

    if cylindrical:
        x = (valid_points[:, 0] * np.cos(valid_points[:, 1])).reshape(-1, 1)
        y = (valid_points[:, 0] * np.sin(valid_points[:, 1])).reshape(-1, 1)
        points = np.hstack((x, y, valid_points[:, 2:]))
    else:
        points = valid_points

    # swap axes
    new_points = np.zeros(points.shape)
    new_points[:, 0] = points[:, 2]
    new_points[:, 1] = points[:, 0]
    new_points[:, 2] = points[:, 1]
    points = new_points

    print(points.shape, labels.shape)

    int_color = LABEL_COLORS[labels]
    point_list = o3d.geometry.PointCloud()
    point_list.points = o3d.utility.Vector3dVector(points)
    point_list.colors = o3d.utility.Vector3dVector(int_color)
    
    vis = o3d.visualization.Visualizer()
    vis.create_window(
    window_name='Segmented Scene',
    width=960,
    height=540,
    left=480,
    top=270)
    vis.get_render_option().background_color = [0.0, 0.0, 0.0]
    vis.get_render_option().point_size = 5
    vis.get_render_option().show_coordinate_frame = True
    
    geometry = o3d.geometry.PointCloud(point_list)
    vis.add_geometry(geometry)
    
    for i in range(500):
        vis.poll_events()
        vis.update_renderer()
        time.sleep(0.005)

    return point_list


In [12]:
# Initialize tensorboard logging
writer = SummaryWriter('runs/lmscnet/'+coordinate_type)

# def update_board():
#     writer.add_image('four_fashion_mnist_images', img_grid)

def plot_training_loss(running_loss, epoch, dataloader, count):
    # ...log the running loss
    print("logging training loss...")
    writer.add_scalar('training loss',
                    running_loss / 1000,
                    epoch * len(dataloader) + count)

    # ...log a Matplotlib Figure showing the model's predictions on a
    # random mini-batch
    # writer.add_figure('predictions vs. actuals',
    #                 plot_classes_preds(net, inputs, labels),
    #                 global_step=epoch * len(dataloader) + count)
    running_loss = 0.0

def plot_miou(pred, ground_truth, epoch, dataloader, count):
    print("logging IoU score...")

    # Compute number of matches for each label
    # labels, gt_counts = torch.unique(ground_truth, return_counts=True, dim=1, sorted=True)
    jaccard = JaccardIndex(num_classes=23)
    iou_score = jaccard(pred.cpu(), ground_truth.cpu())
    
    
    writer.add_scalar('IoU Score',
                    iou_score,
                    epoch * len(dataloader) + count)

def save_model(model, model_name, model_dir):
    '''
    Saves trained pytorch model
    '''
    save_dir = os.path.join(model_dir, model_name)
    torch.save(model.state_dict(), save_dir)

In [13]:
min_dim = torch.tensor(carla_ds._eval_param['min_bound'], device=device) 
max_dim = torch.tensor(carla_ds._eval_param['max_bound'], device=device)

def gen_voxel_grid(x_in, grid_dims, B, T):
    voxel_grid = torch.zeros(B, T, grid_dims[0], grid_dims[1], grid_dims[2], device=device)
    intervals = (max_dim - min_dim) / grid_dims

    for b_i in range(B):
        for t_i in range(T):
            points = x_in[b_i][t_i][:, :3]
            # No points for this frame
            if points.shape[0] == 1:
                continue
            # Valid voxels (make sure to clip)
            valid_point_mask = torch.all((points < max_dim.view(1, 3)) &
                                            (points >= min_dim.view(1, 3)), 1)
            valid_points = points[valid_point_mask, :]
            voxels = torch.floor((valid_points - min_dim) / intervals)
            # Clamp to account for any floating point errors
            voxels = torch.clamp(voxels, torch.zeros(3, device=device).view(1, 3),
                                    torch.tensor(grid_dims, device=device).view(1, 3) - 1)
            print(torch.max(voxels, dim=0))
            # This line is needed to create a mask with number of points, not just binary occupied
            unique_voxels, counts = torch.unique(voxels, return_counts=True, dim=0)
            voxel_grid[b_i, t_i, unique_voxels[:, 0].long(), unique_voxels[:, 1].long(), unique_voxels[:, 2].long()] += counts

    return voxel_grid

In [15]:
epoch_num = 5
class_weights = model.get_class_weights().to("cuda").type(torch.float32)
grid_dims = torch.tensor(carla_ds._grid_size, dtype=torch.int64, device=device)

running_loss = 0.0
output_masked = None
preds_masked = None
criterion = nn.CrossEntropyLoss(weight=class_weights)
for epoch in range(epoch_num):
    running_loss = 0.0
    counter = 0
    for input_data, output, counts in dataloader:
        optimizer.zero_grad()
        # counts = torch.unsqueeze(counts, 1)

        counts = gen_voxel_grid(input_data, grid_dims, B, T)

        # counts = counts.permute(0, 1, 2, 4, 3)
        x = {"3D_OCCUPANCY": counts}
        preds = model(x)["pred_semantic_1_1"]
        preds = torch.permute(preds, (0, 2, 3, 4, 1))
        output = output.permute(0, 1, 3, 2)     
        
        if counter % 1000 == 99:
            probs = nn.functional.softmax(preds, dim = 4)
            # visualize_preds(probs[0].detach().cpu().numpy(), np.asarray(carla_ds._eval_param[‘min_bound’]),
            #                 np.asarray(carla_ds._eval_param[‘max_bound’]), 10, cylindrical=cylindrical)        counts = counts.contiguous().view(-1)
        
        counts = counts.contiguous().view(-1)
        output = output.contiguous().view(-1).long()
        preds = preds.contiguous().view(-1, preds.shape[4])        

        # Criterion requires input (NxC), output (N) dimension
        mask = counts > 0
        output_masked = output[mask]
        preds_masked = preds[mask]
        loss = criterion(preds_masked, output_masked)
        # print(loss)        
        running_loss += loss.item()

        if counter%1000==99:
            plot_miou(preds_masked, output_masked, epoch, dataloader, counter)
            plot_training_loss(running_loss, epoch, dataloader, counter)
            running_loss = 0.0
            
        
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        counter += 1
    # Save model weights at the end of each epoch
    model_weights_name = "model_weights_epoch{epochnum:n}".format(epochnum=epoch)
    model_save_dir = "runs/lmscnet/{ctype:<}/model_weights".format(ctype = coordinate_type)
    save_model(model, model_weights_name, model_save_dir)
    
    print(f'Epoch Num: {epoch} ------ loss: {running_loss}')

torch.return_types.max(
values=tensor([127., 100.,   7.], device='cuda:0'),
indices=tensor([1227, 6368,    0], device='cuda:0'))
torch.return_types.max(
values=tensor([127.,  88.,   7.], device='cuda:0'),
indices=tensor([  117, 19125,     4], device='cuda:0'))
torch.return_types.max(
values=tensor([127., 127.,   7.], device='cuda:0'),
indices=tensor([   8, 3075,    0], device='cuda:0'))
torch.return_types.max(
values=tensor([127., 124.,   7.], device='cuda:0'),
indices=tensor([ 719, 1042,    0], device='cuda:0'))


/home/tigeriv/.local/lib/python3.7/site-packages/ipykernel_launcher.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


RuntimeError: expected scalar type Float but found Double

In [ ]:
# Save Model
model_weights_name = "model_weights"
model_save_dir = "runs/lmscnet/{ctype:<}/model_weights".format(ctype = coordinate_type)
save_model(model, model_weights_name, model_save_dir)

In [ ]:
visualize_preds(probs[5].detach().cpu().numpy(), np.asarray(carla_ds._eval_param['min_bound']), 
                            np.asarray(carla_ds._eval_param['max_bound']), 10, cylindrical=cylindrical)

(195880, 3) (195880,)
[Open3D WARNING] GLFW Error: GLX: Failed to create context: GLXBadFBConfig
[Open3D WARNING] Failed to create window


libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open iris: /usr/lib/dri/iris_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: iris
libGL error: MESA-LOADER: failed to open swrast: /usr/lib/dri/swrast_dri.so: cannot open shared object file: No such file or directory (search paths /usr/lib/x86_64-linux-gnu/dri:\$${ORIGIN}/dri:/usr/lib/dri, suffix _dri)
libGL error: failed to load driver: swrast


AttributeError: 'NoneType' object has no attribute 'background_color'